In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Layer, Multiply
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# Self-Attention Layer
class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[-1],),
                                 initializer='zeros',
                                 trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        attention_scores = K.dot(x, self.W) + self.b
        attention_weights = K.softmax(attention_scores, axis=1)
        output = Multiply()([x, attention_weights])
        return output

# Path to folder containing the Excel files
folder_path = "D:/OneDrive_2024-12-27/Segmented by seperating spindles/Balanced_segmented"

# Load and combine data from all Excel files
all_data = []
all_labels = []
for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        data = df.iloc[:, :-1].values  # Features (all columns except last)
        labels = df.iloc[:, -1].values  # Labels (last column)
        all_data.append(data)
        all_labels.append(labels)

# Combine all files into a single dataset
all_data = np.vstack(all_data)
all_labels = np.hstack(all_labels)

# Normalize the data
all_data = (all_data - np.min(all_data)) / (np.max(all_data) - np.min(all_data))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

# Reshape data for 1D CNN input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the 1D CNN model with Self-Attention
input_layer = Input(shape=(X_train.shape[1], 1))
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
pool1 = MaxPooling1D(pool_size=2)(conv1)
drop1 = Dropout(0.2)(pool1)

conv2 = Conv1D(filters=64, kernel_size=3, activation='relu')(drop1)
pool2 = MaxPooling1D(pool_size=2)(conv2)
drop2 = Dropout(0.2)(pool2)

attention = SelfAttention()(drop2)
flatten = Flatten()(attention)

fc1 = Dense(128, activation='relu')(flatten)
drop3 = Dropout(0.5)(fc1)
output_layer = Dense(2, activation='softmax')(drop3)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
eval_metrics = model.evaluate(X_test, y_test, verbose=0)
print("\nModel Evaluation:")
print(f"Loss: {eval_metrics[0]:.4f}, Accuracy: {eval_metrics[1]:.4f}")

# Predict and print the confusion matrix
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))



Epoch 1/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.4990 - loss: 0.6933 - val_accuracy: 0.5016 - val_loss: 0.6867
Epoch 2/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7748 - loss: 0.4839 - val_accuracy: 0.9121 - val_loss: 0.2625
Epoch 3/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9023 - loss: 0.2788 - val_accuracy: 0.9171 - val_loss: 0.2257
Epoch 4/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9153 - loss: 0.2388 - val_accuracy: 0.9270 - val_loss: 0.2066
Epoch 5/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9207 - loss: 0.2154 - val_accuracy: 0.9197 - val_loss: 0.2093
Epoch 6/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9265 - loss: 0.2050 - val_accuracy: 0.9307 - val_loss: 0.1977
Epoch 7/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9262 - loss: 0.2045 - val_accuracy: 0.9289 - val_loss: 0.1959
Epoch 8/20
706/706 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9327 - loss: 0.1911 - val_accuracy: 0